In [1]:
from FastFitClassifier import FastFitClassifier
import pandas as pd
from tqdm.auto import tqdm

tqdm.pandas()

In [2]:
DEV = True
model_path = "./models/relevance/avsolatorio/GIST-small-Embedding-v0"
# model_path = './models/relevance/avsolatorio/GIST-Embedding-v0'


if DEV:
    model_path=model_path.replace('relevance', 'relevance/dev')
else:
    model_path = model_path.replace('-small','')

In [3]:
def import_labelled_data(path="data/level-1.5/potential/data.json"):
    data = pd.read_json(path, encoding="latin-1")
    return data



print("Loading data...")

test_dataset = import_labelled_data(path="../../../data/level-1.5/potential/data.json", )


print("Data loaded.")

if DEV:
    test_dataset = test_dataset.sample(min(5000,len(test_dataset)))

test_dataset = test_dataset.sample(frac=1).reset_index(drop=True)


test_dataset.info()

Loading data...
Data loaded.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   url           43 non-null     object 
 1   text          43 non-null     object 
 2   relevance     43 non-null     object 
 3   multiclasses  43 non-null     object 
 4   predictions   43 non-null     object 
 5   score         43 non-null     float64
dtypes: float64(1), object(5)
memory usage: 2.1+ KB


In [4]:
test_dataset.head()

,url,text,relevance,multiclasses,predictions,score
0,http://repositorio.unap.edu.pe/bitstream/handl...,UNIVERSIDAD NACIONAL DEL ALTIPLANO ESCUELA DE ...,irrelevant,[],relevant,0.991765
1,https://www.wvlandtrust.org/wp-content/uploads...,Summer 2024 . . . . . . . . . . . . . . . . . ...,irrelevant,[],relevant,0.959468
2,http://dellsadler.com/pdf/november-2010.pdf,"J. I DELL SADLER III, INC. Newsletter - Novemb...",irrelevant,[],relevant,0.987991
3,https://yearofthesalmon.org/wp-content/uploads...,A synthesis of farmed to wild genetic introgre...,irrelevant,[],relevant,0.999995
4,https://www.service-public.pf/dcp/wp-content/u...,e n n e i s n°4 é n y l o [Dossier d’Archéolog...,irrelevant,[],relevant,0.986726


In [5]:
import torch
torch.cuda.empty_cache()

In [6]:
classifier = FastFitClassifier(
    model_path=model_path,
    text_overlap_proportion=0.2,
    device='cuda'
)

Loading model from ./models/relevance/dev/avsolatorio/GIST-small-Embedding-v0 ...
Model loaded.
Tokenizer loaded....


The model 'FastFit' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GemmaForSequenceClassification', 'Gemma2ForSequenceClassif


Classifier pipeline built.


In [7]:

#! on fastfit/modeling.py, line 838, inserting **kwargs to the function arguments
#! it's easier to just silently accept and not use extra tokenizer return values
#! than getting it to stop outputting things like token_type_ids

predicted_data = classifier.evaluate(
    test_dataset,
    metrics=[
        "accuracy",
        "precision",
        "classification-report",
        "specificity",
        "confusion-matrix-mpl",
    ],
    aggregate="majority",
)

  0%|          | 0/43 [00:00<?, ?it/s]

Data chunked.


Files	:   0%|          | 0/43 [00:00<?, ?it/s]

Chunks	:   0%|          | 0/13526 [00:00<?, ?it/s]

Calculated predictions.
Evaluating model...

-----------------------------------------------------

Classification Report: 
              precision    recall  f1-score   support

  irrelevant       1.00      1.00      1.00        43

    accuracy                           1.00        43
   macro avg       1.00      1.00      1.00        43
weighted avg       1.00      1.00      1.00        43

-----------------------------------------------------
Accuracy: 1.0
Precision: 1.0


/root/.local/share/virtualenvs/GreyLiteratureClassifier-eJH_GeT1/lib/python3.10/site-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


ValueError: not enough values to unpack (expected 4, got 1)

In [ ]:
potential = predicted_data[predicted_data["predictions"] == "relevant"]
potential = potential.sort_values(by="score-lv2", ascending=False)


In [ ]:
predicted_data.sort_values(by="score-lv2", ascending=False)['url']

40    https://nast.dost.gov.ph/images/pdf%20files/Pu...
31    https://repositorio.imarpe.gob.pe/bitstream/20...
16    https://www.wrisc.org/_files/ugd/b37ba2_29761e...
6     https://digitalarchive.worldfishcenter.org/bit...
8     https://archiv.zukunftswerk.org/uploads/1/3/1/...
35    https://ieg.worldbankgroup.org/sites/default/f...
29    https://yearofthesalmon.org/wp-content/uploads...
3     https://files.worldwildlife.org/wwfcmsprod/fil...
39    https://zooreach.org/downloads/Education_pkt_c...
42    https://repositorio.unfv.edu.pe/bitstream/hand...
20    https://www.ddcommodities.com/wp-content/uploa...
27    https://yearofthesalmon.org/wp-content/uploads...
38    https://digitalarchive.worldfishcenter.org/bit...
21    https://nast.dost.gov.ph/images/pdf%20files/Pu...
15    https://digitalarchive.worldfishcenter.org/bit...
13    https://yearofthesalmon.org/wp-content/uploads...
7     http://repositorio.unap.edu.pe/bitstream/handl...
2     https://www.service-public.pf/dcp/wp-conte

In [ ]:
potential.head()

,url,text,relevance,multiclasses,predictions,score,score-lv2
11,https://wyocoopunit.org/wp-content/uploads/Hor...,"EVALUATING THE ROLE OF HARVEST, DEMOGRAPHY AND...",irrelevant,[],relevant,0.999352,0.97515


In [ ]:

if DEV:
    from os import makedirs, path
    if not path.exists("../../../data/level-2.5/potential/dev"):
        makedirs("../../../data/level-2.5/potential/dev")
    potential.to_json("../../../data/level-2.5/potential/dev/data.json", orient="records", indent=4)
else:
    potential.to_json("../../../data/level-2.5/potential/data.json", orient="records", indent=4)

In [ ]:
potential['url']

11    https://wyocoopunit.org/wp-content/uploads/Hor...
Name: url, dtype: object